1. 선수단의 평균 월드컵 본선 출전 횟수
    - 데이터 : world_cup_comparisons.csv(1966~2018) / 2002~2022 world_cup_player.csv
    - 나라(연도) - 평균 본선 출전 횟수
        - 한 (나라, 연도)에 이름이 같은 선수가 있나 체크
        - 선수, 나라, 연도 그룹핑해서 각 항목들을 카운팅
        - 카운팅된 숫자를 (나라, 연도) 출전 선수 인원 명수로 나눠서 평균 구하기

2. 모든 월드컵에 대해서 본선 진출 라운드 점수화
    - 데이터 : https://www.kaggle.com/datasets/piterfm/fifa-football-world-cup?select=matches_1930_2022.csv
    - 나라(연도) - 우승, 결승, 4강, 16강, 그룹스테이지 항목들에 대해서 0, 1 처리
        - matches_1930_2022.csv파일에서 round 데이터를 가지고 중복된 값들 처리

##### 트러블 슈팅
1. world_cup_comparisons 파일을 가지고 분류를 해보려 했으나 선수 이름만 컬럼으로 갖고 있는 데이터로는 동명이인 등의 문제점이 있다는 것을 알게 됨<br>
    -> 선수 이름과 함께 선수를 특정지을 수 있는 데이터인 출생연도 데이터도 함께 포함된 데이터를 찾아서 추출<br><br>
2. 찾은 데이터는 1930 ~ 2014 월드컵 데이터가 들어있는 파일이었지만, 1954 월드컵 이전, 2014 월드컵 데이터에 누락과 오류가 많다는 것을 알게 됨<br>
    -> 데이터가 온전히 들어있는 년도들을 nunique() 등을 활용하여 알아내어 1954 ~ 2010 년도 월드컵 데이터만 추출<br><br>
3. 데이터가 온전히 들어있는 연도들의 데이터에도 나라의 이름이 다른 방식으로 표기되어 있거나 한국, 독일, 러시아 등 특수한 상황의 국가들의 경우 통일이 필요하다는 것을 알게 됨<br>
    -> 하나의 나라로 묶을 수 있는 나라의 리스트를 뽑은 다음 현재 나라명으로 바꿔줌<br>
    -> 독일의 경우 1974년에 동독과 서독이 동시 출전을 했어서 동독과 서독을 모두 'Germany'로 매핑할 경우 Germany(1974)의 선수단이 40명이 되지만, 최종 목표가 월드컵 본선 진출 횟수의 평균을 구하는 작업이니 만큼 따로 처리하지 않았음<br><br>
4. 2014 월드컵 이후의 데이터를 찾고 컬럼을 정리하여 앞서 만들었던 2014년 이전의 데이터와 합치는 과정에서 선수의 이름 표기법이나 이름과 성의 순서, 하이픈 등 특수문자의 유무 등으로 본선 출전 횟수의 카운팅이 제대로 작동하지 않았음<br>
    -> 1954 ~ 2010 데이터와, 2014 ~ 2022 데이터 모두 선수의 이름이 담긴 칼럼을 동일한 규칙을 두어 매핑함<br>
    -> 한국, 일본 등 성과 이름의 순서가 바뀌어 있는 경우엔 len(token)을 사용하여 조건문을 만들어 해결함<br><br>

### 1954 ~ 2010 나라별 엔트리의 월드컵 평균 출전 횟수

In [1]:
import pandas as pd, re, unicodedata

old_squads = pd.read_csv('./data/squads_1954_2010.csv')

# 나라 이름 매핑
name_map = {
    # 독일
    'West Germany':'Germany',
    'East Germany':'Germany',
    # 한국
    'Korea Republic':'South Korea',
    # 북한
    'Korea DPR':'North Korea',
    # 세르비아
    'Yugoslavia':'Serbia',
    'FR Yugoslavia':'Serbia',
    'Serbia and Montenegro':'Serbia',
    # 체코슬로바키아
    'Czechoslovakia':'Czechia',
    'Czech Republic':'Czechia',
    'Slovakia':'Czechia',
    # 러시아
    'Soviet Union':'Russia',
}

old_squads['Country'] = old_squads['Country'].str.strip().replace(name_map)

old_squads

,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year
0,1,1GK,Ángel Bossio,(1905-05-05)5 May 1905 (aged 25),NaN,Talleres,Argentina,Argentina,1930.0
1,1,1GK,Juan Botasso,(1908-10-23)23 October 1908 (aged 21),NaN,Quilmes,Argentina,Argentina,1930.0
2,9,4FW,Roberto Cherro,(1907-02-23)23 February 1907 (aged 23),NaN,Boca Juniors,Argentina,Argentina,1930.0
3,4,2DF,Alberto Chividini,(1907-02-23)23 February 1907 (aged 23),NaN,Central Norte Tucumán,Argentina,Argentina,1930.0
4,10,4FW,Attilio Demaría,(1909-03-19)19 March 1909 (aged 21),NaN,Estudiantil Porteño,Argentina,Argentina,1930.0
...,...,...,...,...,...,...,...,...,...
8870,-,4FW,Park Chu-Young,(1985-07-10)10 July 1985 (aged 28),62,Watford,South Korea,England,2014.0
8871,-,4FW,Ji Dong-Won,(1991-05-28)28 May 1991 (aged 23),26,FC Augsburg,South Korea,Germany,2014.0
8872,-,4FW,Lee Keun-Ho,(1985-04-11)11 April 1985 (aged 29),62,Sangju Sangmu,South Korea,South Korea,2014.0
8873,-,4FW,Koo Ja-Cheol,(1989-02-27)27 February 1989 (aged 25),35,Mainz 05,South Korea,Germany,2014.0


In [2]:
# 1954 이전, 2010 이후 데이터 부족으로 1954~2010 데이터만 추출
old_squads = old_squads[(old_squads['Year'] >= 1954) & (old_squads['Year'] <= 2010)]

old_squads

,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year
1182,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0
1183,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0
1184,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0
1185,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0
1186,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0
...,...,...,...,...,...,...,...,...,...
8642,19,3MF,Gonzalo Fierro,(1983-03-21)21 March 1983 (aged 27),16,Flamengo,Chile,Brazil,2010.0
8643,20,3MF,Rodrigo Millar,(1981-11-03)3 November 1981 (aged 28),19,Colo-Colo,Chile,Chile,2010.0
8644,21,3MF,Rodrigo Tello,(1979-10-14)14 October 1979 (aged 30),32,Beşiktaş,Chile,Turkey,2010.0
8645,22,4FW,Esteban Paredes,(1980-08-01)1 August 1980 (aged 29),12,Colo-Colo,Chile,Chile,2010.0


In [3]:
# YYYY-MM-DD 추출
old_squads['DOB'] = old_squads['DOB/Age'].str.extract(r'\((\d{4}-\d{2}-\d{2})\)')

old_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\419583158.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_squads['DOB'] = old_squads['DOB/Age'].str.extract(r'\((\d{4}-\d{2}-\d{2})\)')


,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB
1182,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0,1927-11-27
1183,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0,1929-02-27
1184,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0,1927-05-16
1185,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0,1925-06-09
1186,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0,1932-01-13
...,...,...,...,...,...,...,...,...,...,...
8642,19,3MF,Gonzalo Fierro,(1983-03-21)21 March 1983 (aged 27),16,Flamengo,Chile,Brazil,2010.0,1983-03-21
8643,20,3MF,Rodrigo Millar,(1981-11-03)3 November 1981 (aged 28),19,Colo-Colo,Chile,Chile,2010.0,1981-11-03
8644,21,3MF,Rodrigo Tello,(1979-10-14)14 October 1979 (aged 30),32,Beşiktaş,Chile,Turkey,2010.0,1979-10-14
8645,22,4FW,Esteban Paredes,(1980-08-01)1 August 1980 (aged 29),12,Colo-Colo,Chile,Chile,2010.0,1980-08-01


In [4]:
# 선수 이름 정규화, Player ID 컬럼을 새로 생성함

def normalize_name(name: str) -> str:
    name_nfkd = unicodedata.normalize('NFKD', name)
    name_nfkd = ''.join(c for c in name_nfkd if not unicodedata.combining(c))
    tokens = re.sub(r"[.\-]", " ", name_nfkd).casefold().split()

    return ''.join(tokens)


old_squads['Player_ID'] = old_squads['Player'].apply(normalize_name) + '|' + old_squads['DOB'].fillna('')

old_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\2208768361.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_squads['Player_ID'] = old_squads['Player'].apply(normalize_name) + '|' + old_squads['DOB'].fillna('')


,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB,Player_ID
1182,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0,1927-11-27,castilho|1927-11-27
1183,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0,1929-02-27,djalmasantos|1929-02-27
1184,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0,1927-05-16,niltonsantos|1927-05-16
1185,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0,1925-06-09,brandaozinho|1925-06-09
1186,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0,1932-01-13,pinheiro|1932-01-13
...,...,...,...,...,...,...,...,...,...,...,...
8642,19,3MF,Gonzalo Fierro,(1983-03-21)21 March 1983 (aged 27),16,Flamengo,Chile,Brazil,2010.0,1983-03-21,gonzalofierro|1983-03-21
8643,20,3MF,Rodrigo Millar,(1981-11-03)3 November 1981 (aged 28),19,Colo-Colo,Chile,Chile,2010.0,1981-11-03,rodrigomillar|1981-11-03
8644,21,3MF,Rodrigo Tello,(1979-10-14)14 October 1979 (aged 30),32,Beşiktaş,Chile,Turkey,2010.0,1979-10-14,rodrigotello|1979-10-14
8645,22,4FW,Esteban Paredes,(1980-08-01)1 August 1980 (aged 29),12,Colo-Colo,Chile,Chile,2010.0,1980-08-01,estebanparedes|1980-08-01


In [5]:
# 연도순 정렬 후 누적 본선 경험 계산
old_squads['WC_apps_before'] = (
    old_squads
      .sort_values(['Player_ID', 'Year'])    
      .groupby('Player_ID')
      .cumcount()
)

old_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\4208918279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_squads['WC_apps_before'] = (


,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB,Player_ID,WC_apps_before
1182,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0,1927-11-27,castilho|1927-11-27,0
1183,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0,1929-02-27,djalmasantos|1929-02-27,0
1184,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0,1927-05-16,niltonsantos|1927-05-16,0
1185,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0,1925-06-09,brandaozinho|1925-06-09,0
1186,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0,1932-01-13,pinheiro|1932-01-13,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8642,19,3MF,Gonzalo Fierro,(1983-03-21)21 March 1983 (aged 27),16,Flamengo,Chile,Brazil,2010.0,1983-03-21,gonzalofierro|1983-03-21,0
8643,20,3MF,Rodrigo Millar,(1981-11-03)3 November 1981 (aged 28),19,Colo-Colo,Chile,Chile,2010.0,1981-11-03,rodrigomillar|1981-11-03,0
8644,21,3MF,Rodrigo Tello,(1979-10-14)14 October 1979 (aged 30),32,Beşiktaş,Chile,Turkey,2010.0,1979-10-14,rodrigotello|1979-10-14,0
8645,22,4FW,Esteban Paredes,(1980-08-01)1 August 1980 (aged 29),12,Colo-Colo,Chile,Chile,2010.0,1980-08-01,estebanparedes|1980-08-01,0


In [6]:
# 나라, 연도 그룹핑 후 엔트리별 평균 출전 횟수 열 출력
old_avg_exp = (
    old_squads.groupby(['Country', 'Year'])['WC_apps_before']
    .mean()
    .round(5)
    .reset_index()
)

old_avg_exp

,Country,Year,WC_apps_before
0,Algeria,1982.0,0.00000
1,Algeria,1986.0,0.40909
2,Algeria,2010.0,0.00000
3,Angola,2006.0,0.00000
4,Argentina,1958.0,0.00000
...,...,...,...
329,Uruguay,1986.0,0.00000
330,Uruguay,1990.0,0.22727
331,Uruguay,2010.0,0.00000
332,Wales,1958.0,0.00000


In [7]:
# 출력
old_avg_exp['TeamYear'] = old_avg_exp['Country'].str.strip() + '(' + old_avg_exp['Year'].astype(str) + ')'

old_avg_exp_sorted = (
    old_avg_exp           
    .sort_values(['TeamYear'])
)

result = dict(zip(old_avg_exp_sorted['TeamYear'], old_avg_exp_sorted['WC_apps_before']))

for k, v in result.items():
    print(f"{k} : {v}")

Algeria(1982.0) : 0.0
Algeria(1986.0) : 0.40909
Algeria(2010.0) : 0.0
Angola(2006.0) : 0.0
Argentina(1958.0) : 0.0
Argentina(1962.0) : 0.04545
Argentina(1966.0) : 0.22727
Argentina(1974.0) : 0.0
Argentina(1978.0) : 0.13636
Argentina(1982.0) : 0.59091
Argentina(1986.0) : 0.13636
Argentina(1990.0) : 0.34783
Argentina(1994.0) : 0.45455
Argentina(1998.0) : 0.31818
Argentina(2002.0) : 0.73913
Argentina(2006.0) : 0.21739
Argentina(2010.0) : 0.30435
Australia(1974.0) : 0.0
Australia(2006.0) : 0.0
Australia(2010.0) : 0.43478
Austria(1954.0) : 0.0
Austria(1958.0) : 0.45455
Austria(1978.0) : 0.0
Austria(1982.0) : 0.40909
Austria(1990.0) : 0.04545
Austria(1998.0) : 0.27273
Belgium(1954.0) : 0.0
Belgium(1970.0) : 0.0
Belgium(1982.0) : 0.04545
Belgium(1986.0) : 0.27273
Belgium(1990.0) : 0.5
Belgium(1994.0) : 0.59091
Belgium(1998.0) : 0.59091
Belgium(2002.0) : 0.3913
Bolivia(1994.0) : 0.0
Brazil(1954.0) : 0.0
Brazil(1958.0) : 0.13636
Brazil(1962.0) : 0.54545
Brazil(1966.0) : 0.59091
Brazil(1970.0) :

In [8]:
# 나라(연도) 고유 조합 개수 확인
print('고유 나라(연도) 수  :', old_squads[['Country','Year']].drop_duplicates().shape[0])

# 연도별 참가국 수 확인
year_counts = old_squads.groupby('Year')['Country'].nunique().sort_index()
print(year_counts)

고유 나라(연도) 수  : 334
Year
1954.0    16
1958.0    16
1962.0    16
1966.0    16
1970.0    16
1974.0    15
1978.0    16
1982.0    24
1986.0    24
1990.0    24
1994.0    24
1998.0    32
2002.0    31
2006.0    32
2010.0    32
Name: Country, dtype: int64


### 2014 ~ 2022 나라별 엔트리의 월드컵 평균 출전 횟수

In [9]:
new_squads = pd.read_csv('./data/squads_2014_2022.csv')

kor2eng = {
    '가나': 'Ghana',
    '공화국': 'South Africa',        
    '그리스': 'Greece',
    '나이지리아': 'Nigeria',
    '네덜란드': 'Netherlands',
    '뉴질랜드': 'New Zealand',
    '대한민국': 'South Korea',
    '덴마크': 'Denmark',
    '독일': 'Germany',
    '러시아': 'Russia',
    '멕시코': 'Mexico',
    '모로코': 'Morocco',
    '몬테네그로': 'Serbia',  
    '미국': 'United States',
    '벨기에': 'Belgium',
    '북한': 'North Korea',
    '브라질': 'Brazil',
    '사우디아라비아': 'Saudi Arabia',
    '세네갈': 'Senegal',
    '세르비아': 'Serbia',
    '스웨덴': 'Sweden',
    '스위스': 'Switzerland',
    '스페인': 'Spain',
    '슬로바키아': 'Czechia',
    '슬로베니아': 'Slovenia',
    '아르헨티나': 'Argentina',
    '아이슬란드': 'Iceland',
    '아일랜드': 'Republic of Ireland',
    '알제리': 'Algeria',
    '앙골라': 'Angola',
    '에콰도르': 'Ecuador',
    '온두라스': 'Honduras',
    '우루과이': 'Uruguay',
    '우크라이나': 'Ukraine',
    '웨일스': 'Wales',
    '이란': 'Iran',
    '이집트': 'Egypt',
    '이탈리아': 'Italy',
    '일본': 'Japan',
    '중국': 'China',
    '카메룬': 'Cameroon',
    '카타르': 'Qatar',
    '코스타리카': 'Costa Rica',
    '코트디부아르': "Côte d'Ivoire",
    '콜롬비아': 'Colombia',
    '크로아티아': 'Croatia',
    '터키': 'Turkey',
    '토고': 'Togo',
    '토바고': 'Trinidad and Tobago', 
    '튀니지': 'Tunisia',
    '파나마': 'Panama',
    '파라과이': 'Paraguay',
    '페루': 'Peru',
    '포르투갈': 'Portugal',
    '폴란드': 'Poland',
    '프랑스': 'France',
    '헤르체고비나': 'Bosnia and Herzegovina',  
    '호주': 'Australia',
    '잉글랜드': 'England',
    '체코': 'Czechia',
    '칠레': 'Chile',
    '캐나다': 'Canada'
}

new_squads['Country'] = new_squads['국가'].str.strip().replace(kor2eng)

new_squads

,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,Country
0,0,2002,세네갈,GK,1.0,토니 실바,Tony Sylva,1975.05.17,AS 모나코,Senegal
1,1,2002,세네갈,GK,16.0,오마르 디알로,Omar Diallo,1972.09.28,올랭피크 쿠리브가,Senegal
2,2,2002,세네갈,GK,22.0,칼리두 시소코,Kalidou Cissokho,1978.08.28,ASC 잔다르크,Senegal
3,3,2002,세네갈,DF,2.0,오마르 다프,Omar Daf,1977.02.12,FC 소쇼,Senegal
4,4,2002,세네갈,DF,4.0,파페 말리크 디오프,Pape Malick Diop,1974.12.29,FC 로리앙,Senegal
...,...,...,...,...,...,...,...,...,...,...
4437,4414,2022,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC,South Korea
4438,4415,2022,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크,South Korea
4439,4416,2022,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스,South Korea
4440,4417,2022,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스,South Korea


In [10]:
new_squads = new_squads[(new_squads['년도'] >= 2014) & (new_squads['년도'] <= 2022)]

new_squads

,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,Country
2139,2139,2014,크로아티아,GK,1.0,스티페 플레티코사,Stipe Pletikosa,1979.01.08,FC 로스토프,Croatia
2140,2140,2014,크로아티아,GK,12.0,올리베르 젤레니카,Oliver Zelenika,1993.05.14,NK 로코모티바 자그레브,Croatia
2141,2141,2014,크로아티아,GK,23.0,다니옐 수바시치,Danijel Subašić,1984.10.27,AS 모나코 FC,Croatia
2142,2142,2014,크로아티아,DF,2.0,시메 브르살리코,Šime Vrsaljko,1992.01.10,제노아 CFC,Croatia
2143,2143,2014,크로아티아,DF,3.0,다니옐 프라니치,Danijel Pranjić,1981.12.02,파나티나이코스 FC,Croatia
...,...,...,...,...,...,...,...,...,...,...
4437,4414,2022,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC,South Korea
4438,4415,2022,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크,South Korea
4439,4416,2022,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스,South Korea
4440,4417,2022,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스,South Korea


In [11]:
new_squads['DOB'] = (
    new_squads['생년월일']
      .str.extract(r'(\d{4}[.\-/]\d{2}[.\-/]\d{2})')  
      .iloc[:, 0]                                    
      .str.replace('[./]', '-', regex=True)            
)

new_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\3156596046.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_squads['DOB'] = (


,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,Country,DOB
2139,2139,2014,크로아티아,GK,1.0,스티페 플레티코사,Stipe Pletikosa,1979.01.08,FC 로스토프,Croatia,1979-01-08
2140,2140,2014,크로아티아,GK,12.0,올리베르 젤레니카,Oliver Zelenika,1993.05.14,NK 로코모티바 자그레브,Croatia,1993-05-14
2141,2141,2014,크로아티아,GK,23.0,다니옐 수바시치,Danijel Subašić,1984.10.27,AS 모나코 FC,Croatia,1984-10-27
2142,2142,2014,크로아티아,DF,2.0,시메 브르살리코,Šime Vrsaljko,1992.01.10,제노아 CFC,Croatia,1992-01-10
2143,2143,2014,크로아티아,DF,3.0,다니옐 프라니치,Danijel Pranjić,1981.12.02,파나티나이코스 FC,Croatia,1981-12-02
...,...,...,...,...,...,...,...,...,...,...,...
4437,4414,2022,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC,South Korea,1994-06-30
4438,4415,2022,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크,South Korea,1999-09-20
4439,4416,2022,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스,South Korea,1999-09-12
4440,4417,2022,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스,South Korea,1998-01-25


In [12]:
# 이름 정규화
new_squads['Player_ID'] = new_squads['로마자 성명'].apply(normalize_name) + '|' + new_squads['DOB'].fillna('')

new_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\295442269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_squads['Player_ID'] = new_squads['로마자 성명'].apply(normalize_name) + '|' + new_squads['DOB'].fillna('')


,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,Country,DOB,Player_ID
2139,2139,2014,크로아티아,GK,1.0,스티페 플레티코사,Stipe Pletikosa,1979.01.08,FC 로스토프,Croatia,1979-01-08,stipepletikosa|1979-01-08
2140,2140,2014,크로아티아,GK,12.0,올리베르 젤레니카,Oliver Zelenika,1993.05.14,NK 로코모티바 자그레브,Croatia,1993-05-14,oliverzelenika|1993-05-14
2141,2141,2014,크로아티아,GK,23.0,다니옐 수바시치,Danijel Subašić,1984.10.27,AS 모나코 FC,Croatia,1984-10-27,danijelsubasic|1984-10-27
2142,2142,2014,크로아티아,DF,2.0,시메 브르살리코,Šime Vrsaljko,1992.01.10,제노아 CFC,Croatia,1992-01-10,simevrsaljko|1992-01-10
2143,2143,2014,크로아티아,DF,3.0,다니옐 프라니치,Danijel Pranjić,1981.12.02,파나티나이코스 FC,Croatia,1981-12-02,danijelpranjic|1981-12-02
...,...,...,...,...,...,...,...,...,...,...,...,...
4437,4414,2022,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC,South Korea,1994-06-30,kwonchanghoon|1994-06-30
4438,4415,2022,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크,South Korea,1999-09-20,jeongwooyeong|1999-09-20
4439,4416,2022,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스,South Korea,1999-09-12,songminkyu|1999-09-12
4440,4417,2022,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스,South Korea,1998-01-25,choguesung|1998-01-25


In [13]:
# old, new 데이터프레임 합치기
new_squads['Year'] = new_squads['년도']

all_squads = pd.concat([old_squads, new_squads], ignore_index=True)
all_squads

C:\Users\vibe\AppData\Local\Temp\ipykernel_21060\1837144880.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_squads['Year'] = new_squads['년도']


,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB,...,WC_apps_before,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀
0,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0,1927-11-27,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0,1929-02-27,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0,1927-05-16,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0,1925-06-09,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0,1932-01-13,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9762,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1994-06-30,...,NaN,4414.0,2022.0,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC
9763,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1999-09-20,...,NaN,4415.0,2022.0,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크
9764,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1999-09-12,...,NaN,4416.0,2022.0,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스
9765,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1998-01-25,...,NaN,4417.0,2022.0,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스


In [14]:
if 'WC_apps_before' in all_squads.columns:
    all_squads = all_squads.drop(columns=['WC_apps_before'])

all_squads['WC_apps_before'] = (
    all_squads
      .sort_values(['Player_ID', 'Year'])
      .groupby('Player_ID')
      .cumcount()
)

all_squads

,No,Pos,Player,DOB/Age,Caps,Club,Country,ClubCountry,Year,DOB,...,Unnamed: 0,년도,국가,포지션,등번호,한글 성명,로마자 성명,생년월일,소속팀,WC_apps_before
0,1,1GK,Castilho,(1927-11-27)27 November 1927 (aged 26),13,Fluminense,Brazil,Brazil,1954.0,1927-11-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,2DF,Djalma Santos,(1929-02-27)27 February 1929 (aged 25),14,Portuguesa,Brazil,Brazil,1954.0,1929-02-27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3,2DF,Nílton Santos,(1927-05-16)16 May 1927 (aged 27),18,Botafogo,Brazil,Brazil,1954.0,1927-05-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4,2DF,Brandãozinho,(1925-06-09)9 June 1925 (aged 29),12,Portuguesa,Brazil,Brazil,1954.0,1925-06-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5,3MF,Pinheiro,(1932-01-13)13 January 1932 (aged 22),13,Fluminense,Brazil,Brazil,1954.0,1932-01-13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9762,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1994-06-30,...,4414.0,2022.0,대한민국,MF,22.0,권창훈,KWON Changhoon,1994.06.30.,김천 상무 FC,0
9763,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1999-09-20,...,4415.0,2022.0,대한민국,MF,25.0,정우영,JEONG Wooyeong,1999.09.20.,SC 프라이부르크,0
9764,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1999-09-12,...,4416.0,2022.0,대한민국,MF,26.0,송민규,SONG Minkyu,1999.09.12.,전북 현대 모터스,0
9765,NaN,NaN,NaN,NaN,NaN,NaN,South Korea,NaN,2022.0,1998-01-25,...,4417.0,2022.0,대한민국,FW,9.0,조규성,CHO Guesung,1998.01.25.,전북 현대 모터스,0


In [15]:
avg_exp = (
    all_squads.groupby(['Country', 'Year'])['WC_apps_before']
    .mean()
    .round(5)
    .reset_index()
    .sort_values(['Country', 'Year'])
)

avg_exp['TeamYear'] = avg_exp['Country'] + '(' + avg_exp['Year'].astype(str) + ')'
result = dict(zip(avg_exp['TeamYear'], avg_exp['WC_apps_before']))

for k, v in result.items():
    print(f"{k} : {v}")

Algeria(1982.0) : 0.0
Algeria(1986.0) : 0.40909
Algeria(2010.0) : 0.0
Algeria(2014.0) : 0.17391
Angola(2006.0) : 0.0
Argentina(1958.0) : 0.0
Argentina(1962.0) : 0.04545
Argentina(1966.0) : 0.22727
Argentina(1974.0) : 0.0
Argentina(1978.0) : 0.13636
Argentina(1982.0) : 0.59091
Argentina(1986.0) : 0.13636
Argentina(1990.0) : 0.34783
Argentina(1994.0) : 0.45455
Argentina(1998.0) : 0.31818
Argentina(2002.0) : 0.73913
Argentina(2006.0) : 0.21739
Argentina(2010.0) : 0.30435
Argentina(2014.0) : 0.52174
Argentina(2018.0) : 0.65217
Argentina(2022.0) : 0.5
Australia(1974.0) : 0.0
Australia(2006.0) : 0.0
Australia(2010.0) : 0.43478
Australia(2014.0) : 0.26087
Australia(2018.0) : 0.43478
Australia(2022.0) : 0.38462
Austria(1954.0) : 0.0
Austria(1958.0) : 0.45455
Austria(1978.0) : 0.0
Austria(1982.0) : 0.40909
Austria(1990.0) : 0.04545
Austria(1998.0) : 0.27273
Belgium(1954.0) : 0.0
Belgium(1970.0) : 0.0
Belgium(1982.0) : 0.04545
Belgium(1986.0) : 0.27273
Belgium(1990.0) : 0.5
Belgium(1994.0) : 0.5

In [16]:
# result -> csv

df = pd.DataFrame(
    list(result.items()),          
    columns=['TeamYear', 'AvgApps']
)

df[['Team','Year']] = df['TeamYear'].str.extract(r'^(.*?)\((\d{4})')
df['Year'] = pd.to_numeric(df['Year'], errors='coerce').astype('Int64')

df = df[['Team','Year','AvgApps']]
df.to_csv('wc_avg_apps_1954_2022.csv', index=False, encoding='utf-8-sig')
